In [2]:
import pandas as pd 
from file_struct import locate_data_, correct_slash
from data_prep import winter_weather
from waterway import list_stuwvak

d = locate_data_()
s = correct_slash()

In [3]:
weather_data = pd.read_csv(locate_data_() + correct_slash() + "full_weather.csv")
weather_data.tail()

,Unnamed: 0,Day,Min temp,Max temp,Avg temp,Wind speed,Precipitation,Date
1298,1303,27,15.8,22.3,18.7,6.1,2.6,2021-07-27
1299,1304,28,14.4,22.1,17.9,10.6,3.2,2021-07-28
1300,1305,29,12.9,21.6,17.3,12.7,0.8,2021-07-29
1301,1306,30,10.3,23.9,17.1,8.2,0.8,2021-07-30
1302,1307,31,13.4,21.1,17.1,12.9,3.0,2021-07-31


In [4]:
weather = winter_weather(locate_data_() + correct_slash())
weather["TIME"] = pd.to_datetime(weather["Date"])
weather.drop(columns=["month", "Date"], inplace = True)

In [5]:
lst_compartments = list_stuwvak('Leijgraaf')
df = pd.read_csv(locate_data_() + correct_slash() + "feature_tables" + correct_slash() + lst_compartments[0] + "_feature_table.csv")
df["TIME"] = pd.to_datetime(pd.to_datetime(df["TIME"]).dt.date)
ready_data = pd.merge(weather, df, on="TIME")
ready_data.drop(columns=["Unnamed: 0", "Day", "STUWVAK"], inplace=True)

In [6]:
from baseline import get_winter_data

test = get_winter_data(lst_compartments[1])
test.head(3)

,Min temp,Max temp,Avg temp,Wind speed,Precipitation,TIME,VERSCHIL,Q
0,4.9,9.0,6.8,15.4,11.8,2018-01-01,0.0,0.646808
1,4.6,8.2,6.2,12.2,4.4,2018-01-02,0.0,0.725779
2,6.4,11.2,8.9,26.7,16.6,2018-01-03,0.0,1.014376


## Exploring the weir compartment's statistics
Looking at compartments in Leijgraaf to identify good/bad data quality compartments

In [7]:
from file_struct import locate_data_, correct_slash
data_path = locate_data_()
s = correct_slash()
compartments = list_stuwvak("Leijgraaf")


In [8]:

comp = compartments[0]
df = pd.read_csv(data_path + s + "feature_tables" + s + comp + "_feature_table.csv")
verschil_stats = df["VERSCHIL"].describe()
q_stats = df["Q"].describe()
df[df["VERSCHIL"] <= 0].shape[0]
verschil_stats[1]
for i in range(1,4):
    print(i)

1
2
3


In [9]:
dct_stats = {"Stuwvak":compartments, "v_mean":[], "v_std":[], "v_negatives":[], "q_mean":[], "q_std":[], "q_negatives":[]}

for comp in compartments:
    df = pd.read_csv(data_path + s + "feature_tables" + s + comp + "_feature_table.csv")
    verschil_stats = df["VERSCHIL"].describe()
    q_stats = df["Q"].describe()
    dct_stats["v_mean"].append(verschil_stats["mean"])
    dct_stats["v_std"].append(verschil_stats["std"])
    dct_stats["v_negatives"].append(df[df["VERSCHIL"] <= 0].shape[0])
    dct_stats["q_mean"].append(q_stats["mean"])
    dct_stats["q_std"].append(q_stats["std"])
    dct_stats["q_negatives"].append(df[df["Q"] <= 0].shape[0])


df_summary = pd.DataFrame(dct_stats)
df_summary

,Stuwvak,v_mean,v_std,v_negatives,q_mean,q_std,q_negatives
0,211VEL_211N,0.048025,0.123444,244,0.555998,0.413567,39
1,211N_211M,0.175096,0.146955,187,0.636628,0.221624,0
2,211M_211L,0.204599,0.129364,74,2.556836,2.565410,0
3,211L_211K,0.200540,0.133870,17,0.553873,0.610101,408
4,211K_211J,0.115033,0.230162,215,0.893540,0.368289,0
5,211J_211I,0.142565,0.089524,22,0.496124,0.372252,5
6,211I_211H,0.088524,0.039632,0,0.992063,0.350311,0
7,211H_211G,-0.004645,0.021240,814,0.903305,0.466546,3
8,211G_211F,0.064830,0.041800,0,0.508849,0.403207,19
9,211F_211E,0.014924,0.039028,647,0.531705,0.643727,67


In [10]:
dct_stats = {"Compartment":[i for i in compartments for _ in range(3)], "value":[], "type":[]}
dct_i = {1:"mean", 2:"std", 3:"negative_values"}

for comp in compartments:
    df = pd.read_csv(data_path + s + "feature_tables" + s + comp + "_feature_table.csv")
    verschil_stats = df["VERSCHIL"].describe()
    for i in range(1,4):
        dct_stats["value"].append(verschil_stats[i])
        dct_stats["type"].append(dct_i[i])

pd.DataFrame(dct_stats).head(3)

,Compartment,value,type
0,211VEL_211N,0.048025,mean
1,211VEL_211N,0.123444,std
2,211VEL_211N,-0.523824,negative_values


# Find a way to visualize correctness of winter baseline

In [11]:
from data_prep import prep_single_df
dct = {"correlation":[], "year":[], "ratio_innacurate":[]}
comp = "211C_211B"
df = prep_single_df(comp)
winter_months = [10, 11, 12, 1, 2]
df_winter = df[df["MONTH"].isin(winter_months)]

for yr in df_winter["YEAR"].unique():
    df_yearly = df_winter[df_winter["YEAR"] == yr]
    corr = df_yearly.Q.corr(df_yearly.VERSCHIL)
    nr_datapoints = df_yearly.shape[0]
    inaccuracy = df_yearly[df_yearly["VERSCHIL"] < 0.05].shape[0]/nr_datapoints
    dct["correlation"].append(corr)
    dct["year"].append(yr)
    dct["ratio_innacurate"].append(inaccuracy)

results = pd.DataFrame(dct)
results

,correlation,year,ratio_innacurate
0,0.852362,2018,0.933333
1,-0.038034,2019,1.000000
2,0.870928,2020,0.986842
3,0.911039,2021,1.000000


In [12]:
import plotly.graph_objects as go 

fig = go.Figure(data=[
    go.Bar(name="corr", x=results.year, y=results.correlation,text="correlation", textposition="auto", textangle=40),
    go.Bar(name="inaccuracy", x=results.year, y=results.ratio_innacurate, text="% inaccurate", textposition="auto", textangle=40)
])

fig.update_layout(barmode="group")
fig.show()

In [13]:
from baseline import winter_dates

windates = winter_dates(as_datetime=True)
#df[(df["TIME"] >= windates[0][0]) & (df["TIME"] <= windates[0][1])].head(3)

In [15]:
df

,TIME,STUWVAK,VERSCHIL,Q,YEAR,MONTH
0,2018-01-02 00:00:00+00:00,211C_211B,0.069323,4.385643,2018,1
1,2018-01-03 00:00:00+00:00,211C_211B,0.094958,6.412544,2018,1
2,2018-01-04 00:00:00+00:00,211C_211B,0.072458,4.837835,2018,1
3,2018-01-05 00:00:00+00:00,211C_211B,0.072646,2.657719,2018,1
4,2018-01-06 00:00:00+00:00,211C_211B,0.065156,2.268533,2018,1
...,...,...,...,...,...,...
1287,2021-07-12 00:00:00+00:00,211C_211B,-0.000740,0.213985,2021,7
1288,2021-07-13 00:00:00+00:00,211C_211B,0.003635,0.403506,2021,7
1289,2021-07-14 00:00:00+00:00,211C_211B,0.005844,0.409719,2021,7
1290,2021-07-15 00:00:00+00:00,211C_211B,0.016312,2.577480,2021,7


In [17]:
df["WINTER"] = df.apply(lambda x: x["YEAR"] if x["MONTH"] < 7 else x["YEAR"]+1, axis=1)
df

,TIME,STUWVAK,VERSCHIL,Q,YEAR,MONTH,WINTER
0,2018-01-02 00:00:00+00:00,211C_211B,0.069323,4.385643,2018,1,2018
1,2018-01-03 00:00:00+00:00,211C_211B,0.094958,6.412544,2018,1,2018
2,2018-01-04 00:00:00+00:00,211C_211B,0.072458,4.837835,2018,1,2018
3,2018-01-05 00:00:00+00:00,211C_211B,0.072646,2.657719,2018,1,2018
4,2018-01-06 00:00:00+00:00,211C_211B,0.065156,2.268533,2018,1,2018
...,...,...,...,...,...,...,...
1287,2021-07-12 00:00:00+00:00,211C_211B,-0.000740,0.213985,2021,7,2022
1288,2021-07-13 00:00:00+00:00,211C_211B,0.003635,0.403506,2021,7,2022
1289,2021-07-14 00:00:00+00:00,211C_211B,0.005844,0.409719,2021,7,2022
1290,2021-07-15 00:00:00+00:00,211C_211B,0.016312,2.577480,2021,7,2022


# Garbage code from the dashboard

Code that may be put back, but now removed for more clarity

In [ ]:

    st.markdown("Summary statistics of Leijgraaf stream")

    df_summary = get_summary_stats(True, True)
    df_summary = df_summary[df_summary["type"] != "negative_values"]
    fig3 = px.violin(df_summary, x="type", y="value")
    st.plotly_chart(fig3)